# 01 — Images & CCD Calibration (Bias / Dark / Flat)

## What you’ll learn
- What bias, dark, and flat frames correct for
- How to build “master” calibration frames (median combine)
- A practical calibration pipeline
- Simple sigma-clipping to reject outliers (cosmic rays)

We’ll use **synthetic CCD frames** so the notebook runs anywhere.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

np.random.seed(0)

def imshow(img, title=None, vmin=None, vmax=None):
    plt.figure(figsize=(6,5))
    plt.imshow(img, origin="lower", vmin=vmin, vmax=vmax)
    plt.colorbar(label="counts")
    if title: plt.title(title)
    plt.xlabel("x [pix]")
    plt.ylabel("y [pix]")
    plt.tight_layout()
    plt.show()


## 1) Build a synthetic “truth” scene

Think of this as the *real sky signal* in electron counts:
- a smooth background (sky glow)
- a few stars (Gaussian PSFs)


In [ ]:
def make_star_field(ny=200, nx=200, nstars=30, fwhm=3.0, sky=100.0):
    y, x = np.mgrid[0:ny, 0:nx]
    img = np.full((ny, nx), sky, dtype=float)
    sigma = fwhm / 2.355

    for _ in range(nstars):
        x0 = np.random.uniform(10, nx-10)
        y0 = np.random.uniform(10, ny-10)
        flux = np.random.uniform(2_000, 20_000)
        psf = flux * np.exp(-((x-x0)**2 + (y-y0)**2)/(2*sigma**2))
        img += psf
    return img

truth = make_star_field()
imshow(truth, "Truth image (sky + stars)")


## 2) Simulate CCD systematics

We’ll create:
- **Bias**: readout offset pattern (adds counts)
- **Dark current**: thermal electrons (adds counts scaled by exposure time)
- **Flat field**: pixel-to-pixel sensitivity variation (multiplies the image)
- **Read noise + shot noise**
- **Cosmic rays**: rare bright spikes


In [ ]:
def simulate_bias(ny, nx, level=500, pattern_amp=20, read_noise=5):
    y, x = np.mgrid[0:ny, 0:nx]
    pattern = pattern_amp * np.sin(2*np.pi*x/nx) + (pattern_amp/2)*np.cos(2*np.pi*y/ny)
    bias = level + pattern + np.random.normal(0, read_noise, size=(ny,nx))
    return bias

def simulate_dark(ny, nx, dark_e_per_s=0.2, exptime=60.0, hot_pix_frac=0.001):
    dark = np.random.poisson(dark_e_per_s*exptime, size=(ny,nx)).astype(float)
    # hot pixels
    nhot = int(hot_pix_frac*ny*nx)
    if nhot > 0:
        idx = np.random.choice(ny*nx, nhot, replace=False)
        dark.flat[idx] += np.random.uniform(50, 500, size=nhot)
    return dark

def simulate_flat(ny, nx, vignetting=0.15, pix_var=0.02):
    y, x = np.mgrid[0:ny, 0:nx]
    r = np.sqrt((x-nx/2)**2 + (y-ny/2)**2) / (0.5*min(nx,ny))
    vign = 1.0 - vignetting * np.clip(r, 0, 1)**2
    pix = np.random.normal(1.0, pix_var, size=(ny,nx))
    flat = vign * pix
    # normalize like real flats
    return flat / np.median(flat)

def add_cosmic_rays(img, n=25, strength=(20_000, 80_000)):
    out = img.copy()
    ny, nx = out.shape
    for _ in range(n):
        x0 = np.random.randint(0, nx)
        y0 = np.random.randint(0, ny)
        out[y0, x0] += np.random.uniform(*strength)
    return out

ny, nx = truth.shape
bias_true = simulate_bias(ny, nx)
dark_true = simulate_dark(ny, nx, exptime=60)
flat_true = simulate_flat(ny, nx)

imshow(bias_true, "Bias (true)")
imshow(dark_true, "Dark (true)", vmax=np.percentile(dark_true, 99))
imshow(flat_true, "Flat (true)", vmin=0.8, vmax=1.2)


## 3) Make “raw” science frames

Raw CCD model (simplified):
\[
\text{raw} = (\text{truth} \times \text{flat}) + \text{bias} + \text{dark} + \text{noise}
\]


In [ ]:
def simulate_raw_science(truth, exptime=60.0, read_noise=8.0, gain=1.0):
    ny, nx = truth.shape
    bias = simulate_bias(ny, nx, read_noise=read_noise)
    dark = simulate_dark(ny, nx, exptime=exptime)
    flat = simulate_flat(ny, nx)
    # apply flat (multiplicative sensitivity)
    signal = truth * flat

    # shot noise (Poisson)
    poisson = np.random.poisson(np.clip(signal, 0, None)).astype(float)
    raw = poisson + bias + dark

    # read noise
    raw += np.random.normal(0, read_noise, size=(ny,nx))
    # cosmic rays
    raw = add_cosmic_rays(raw, n=20)
    return raw, bias, dark, flat

raw, bias_meas, dark_meas, flat_meas = simulate_raw_science(truth)
imshow(raw, "Raw science frame", vmax=np.percentile(raw, 99.5))


## 4) Build master calibration frames

In practice you take **many** bias/dark/flat frames and combine them (median is robust).


In [ ]:
def median_stack(frames):
    return np.median(np.stack(frames, axis=0), axis=0)

# simulate calibration sets
nbias, ndark, nflat = 25, 15, 15
bias_frames = [simulate_bias(ny,nx,read_noise=8) for _ in range(nbias)]
dark_frames = [simulate_dark(ny,nx,exptime=60) + np.random.normal(0, 3, size=(ny,nx)) for _ in range(ndark)]

# Flats: usually bright images of a uniform source + bias + (maybe) dark
flat_illum = 30_000 * flat_true  # pretend lamp illumination scaled by flat response
flat_frames = []
for _ in range(nflat):
    bias = simulate_bias(ny,nx,read_noise=8)
    noise = np.random.poisson(np.clip(flat_illum,0,None)).astype(float)
    flat_frames.append(noise + bias)

master_bias = median_stack(bias_frames)
master_dark = median_stack(dark_frames)
master_flat_raw = median_stack(flat_frames)

# typical flat processing: subtract bias, then normalize
master_flat = (master_flat_raw - master_bias)
master_flat /= np.median(master_flat)

imshow(master_bias, "Master bias")
imshow(master_dark, "Master dark", vmax=np.percentile(master_dark, 99))
imshow(master_flat, "Master flat (normalized)", vmin=0.8, vmax=1.2)


## 5) Calibrate the science frame

Pipeline:
1. subtract master bias
2. subtract master dark (scaled to exposure time if needed)
3. divide by master flat


In [ ]:
def calibrate(raw, master_bias, master_dark, master_flat, exptime=60.0, dark_exptime=60.0):
    dark_scaled = master_dark * (exptime / dark_exptime)
    cal = (raw - master_bias - dark_scaled) / master_flat
    return cal

cal = calibrate(raw, master_bias, master_dark, master_flat)
imshow(cal, "Calibrated image", vmax=np.percentile(cal, 99.5))


### How close did we get to the truth?

We can compare calibrated vs. truth.  
There will still be differences due to noise + imperfect masters.


In [ ]:
resid = cal - truth
imshow(resid, "Residual (calibrated − truth)", vmin=np.percentile(resid, 2), vmax=np.percentile(resid, 98))

print("Residual stats:")
print("  mean:", np.mean(resid))
print("  std :", np.std(resid))


## 6) Sigma-clipping stack (cosmic-ray robust)

When you have multiple science frames, you often stack them:
- improve SNR by ~sqrt(N)
- reject cosmic rays using sigma-clipping


In [ ]:
def sigma_clip_stack(frames, sigma=4.0, iters=3):
    data = np.stack(frames, axis=0).astype(float)
    mask = np.zeros_like(data, dtype=bool)

    for _ in range(iters):
        # compute stats ignoring masked
        m = np.ma.array(data, mask=mask)
        med = np.ma.median(m, axis=0)
        std = np.ma.std(m, axis=0)
        # mask outliers
        mask = mask | (np.abs(data - med) > sigma*std)
    # final mean of unmasked
    m = np.ma.array(data, mask=mask)
    return np.ma.mean(m, axis=0).filled(np.nan)

# make a small stack of raw frames
raws = [simulate_raw_science(truth)[0] for _ in range(10)]
cals = [calibrate(r, master_bias, master_dark, master_flat) for r in raws]

stack_simple = np.mean(np.stack(cals), axis=0)
stack_clip = sigma_clip_stack(cals, sigma=4.0, iters=3)

imshow(stack_simple, "Stack: simple mean", vmax=np.percentile(stack_simple, 99.5))
imshow(stack_clip, "Stack: sigma-clipped mean", vmax=np.percentile(stack_clip, 99.5))


## Try it
- Increase the number of cosmic rays and see how stacking behaves.
- Increase the number of frames and confirm the noise decreases ~sqrt(N).
- Change the flat vignetting strength and see how bad flats distort photometry.

Next notebook: **02 — Photometry Basics**
